Create a model using built-in library from Pytorch.
This code closely follows nn_tutorial notebook.

In [7]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
!pwd
os.chdir('gdrive/My Drive/PyHack2019/')
!pwd
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content
/content/gdrive/My Drive/PyHack2019
mfcc_noise_attr.pkl  mix_chroma_tar.pkl  model	 TEST_S_X_ATT.pkl
mfcc_noise_tar.pkl   mix_mfcc_attr.pkl	 result  TEST_S_X_TAR.pkl
mix_chroma_attr.pkl  mix_mfcc_tar.pkl	 TEST


In [8]:
import csv, torch, math, os, pickle
from torch import nn
from torch import optim
import torch.nn.functional as F

global chroma_shape, epochs, train_bs, validate_bs, lr, n_class

if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'
  
print(device)

cuda


In [0]:
## read attr and tar in .pkl files
## Param: ratio = training size to the sample size. 
##         E.g. 0.7 means using 70% of the samples as training 
##              sample and the rest as validation sample. 
## Return: (1) a list of padded attr arrays
##         (2) a list of paddrd tar arrays
def read_data(ratio):
    
    assert (ratio > 0 and ratio < 1), 'Invalid ratio'
    
    
    att_file = open(r'mfcc_noise_attr.pkl', 'rb')
    x_train = pickle.load(att_file)  
    att_file.close()
    
    tar_file = open(r'mfcc_noise_tar.pkl', 'rb')
    str_y_train = pickle.load(tar_file)
    tar_file.close()
    
    y = torch.tensor([0 if s == 's' else (1 if s == 'x' else 2) for s in str_y_train])
    assert (len(x_train) == len(y)), 'Unequal sample lengths'
    
    #print([(x,s) for x, s in zip(str_y_train, y_train)])
    
    ## need .float so that it has the same type as weights in the model
    #return torch.tensor(x_train).float(), y_train
    #print(x_train[0].shape, x_train[1].shape, x_train[2].shape)
    #print(x_train[len(x_train)-2].shape)
    x = torch.tensor(x_train).float() 
    divider = int(len(y)*ratio)
    
    
    x_train = x[:divider, :, :]
    y_train = y[:divider]
    
    print(x_train.shape, x.shape)

    x_validate = x[divider:, :, :]
    y_validate = y[divider:]

    assert (x_train.shape[0] + x_validate.shape[0] == len(y_train) + len(y_validate)), 'Lengths do not add up'

    return x_train, y_train, x_validate, y_validate


In [10]:
x_train, y_train, x_validate, y_validate = read_data(0.7)

x_train = x_train.to(device)
y_train = y_train.to(device)
x_validate = x_validate.to(device)
y_validate = y_validate.to(device)

print (x_train.device)

torch.Size([11709, 13, 239]) torch.Size([16728, 13, 239])
cuda:0


In [7]:
x_train.shape

torch.Size([11709, 13, 239])

In [0]:
def accuracy(out, yb):
    ##get the index with the max
    preds = torch.argmax(out, dim = 1)
    return (preds == yb).float().mean()


In [0]:
## Get the shape of a padded instance for model construction
chroma_shape = x_train[0].shape

#epochs = 50
#train_bs = 20
#validate_bs = train_bs*2
n_train = x_train.shape[0]
n_validate = x_validate.shape[0]
n_class = 3

loss_func = F.cross_entropy




Since there are three classes, we set D_out to 3. n is total number of instances and c is the number of attributes in each instance. We use a loss function from torch.nn.functional.

In [0]:
## 13*239
class SoundRecognition_CNN(nn.Module):
  
    def __init__(self, dropout_rate):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=0)
        self.conv3 = nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=0)
        
        self.dropout = nn.Dropout(p = dropout_rate)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = torch.nn.Linear(16*3*116, n_class)

    def forward(self, xb):
        #print(type(xb), len(xb))
        
        ## input of size (bs, 1, 12, 60)
        xb = xb.view(-1, 1, chroma_shape[0], chroma_shape[1])
        #print('xb shape', xb.shape)
        
        ## (bs, 1, 12, 60) >> (bs, 16, 10, 58)
        xb = self.dropout(F.relu(self.conv1(xb)))
        #print('---xb shape2', xb.shape)
        
        ## (bs, 16, 10, 58) >> (bs, 16, 8, 56)
        xb = self.dropout(F.relu(self.conv2(xb)))
        #print('---xb shape3', xb.shape)
        
        ## (bs, 16, 8, 56) >> (bs, 16, 6, 54)
        xb = self.dropout(F.relu(self.conv3(xb)))
        #print('---xb shape4', xb.shape)
        
        ## (bs, 16, 6, 54) >> (bs, 16, 3, 27)
        xb = self.pool(xb)
        #print('---xb shape5', xb.shape)
        #print('sh', xb.shape)
        ## reshape for fully connected
        #xb = xb.view(-1, 16*3*27)
        xb = xb.view(-1, 16*3*116)
        
        ## (bs, 16*3*27) >> (bs, 3)
        xb = self.fc1(xb)
        #print('---xb shape6', xb.shape)
        #print('=====', xb.shape)
        return xb.view(-1, xb.size(1))

## Get the model and optim object that will be used to update model parameters
def get_model(dropout_rate, weight_decay, lr):
    model = SoundRecognition_CNN(dropout_rate)
    return model, optim.Adam(model.parameters(), weight_decay = weight_decay, lr = lr)

In [0]:
def validate(trained_model, validate_bs, n_validate):
    trained_model.eval()
    loss = []
    acc = []
    with torch.no_grad():

        for i in range((n_validate - 1) // validate_bs + 1):
            start_i = i * validate_bs
            end_i = start_i + validate_bs
            #print(xb.shape)
            xb = x_validate[start_i:end_i, :, :]
            yb = y_validate[start_i:end_i]
            pred = trained_model(xb)
            loss.append(loss_func(pred, yb))
            acc.append(accuracy(pred, yb))

        #print(loss)
        valid_loss = sum(loss)
    return valid_loss, sum(acc)/len(acc)

In [0]:
def fit(epochs = 100, 
        train_bs = 10, 
        n_train = n_train, 
        validate_bs = 20, 
        n_validate = n_validate,
        dropout_rate = 0.5, 
        weight_decay = 1e-6, 
        lr = 1e-4,
        tolerance = 0.05):
  
  
  model, opt = get_model(dropout_rate = dropout_rate, weight_decay = weight_decay, lr = lr)
  model.to(device)
  
  ## to save training progress
  text_file = ''
  prev_small_vl = 10000000
  for epoch in range(epochs):

      #print('Training')
      model.train()
      for i in range((n_train - 1) // train_bs + 1):

          start_i = i * train_bs
          end_i = start_i + train_bs
          xb = x_train[start_i:end_i, :, :]
          yb = y_train[start_i:end_i]
          pred = model(xb)
          #print('pred: ', pred, ' | yb: ', yb)
          loss = loss_func(pred, yb)

          loss.backward()
          opt.step()
          opt.zero_grad()

      valid_loss, acc = validate(model, validate_bs, n_validate)
      
      ##early stop
      if valid_loss < prev_small_vl:
        prev_small_vl = valid_loss
      elif valid_loss > prev_small_vl*(1+tolerance):
        print('Epoch: ', epoch, ' == Early stop due to an increase in validation loss')
        break
      

      if (epoch + 1)% 10 == 0:
          print('Epoch: ', epoch + 1, ' | Loss: ', valid_loss, ' | Accuracy: ', acc)
          text_file = text_file + 'Epoch: '+ str(epoch + 1) + ' | Loss: ' + str(valid_loss) + ' | Accuracy: ' + str(acc) + '\n'
  text_file = str(model) + '\n\n' + text_file
  print('Train Finished')
  return model, text_file

In [0]:
# m, t = fit(epochs = 20, 
#     train_bs = 10, 
#     n_train = n_train, 
#     validate_bs = 20, 
#     n_validate = n_validate,
#     dropout_rate = 0.5, 
#     weight_decay = 1e-6, 
#     lr = 1e-4)

In [0]:
def getName(epochs, dropout_rate, weight_decay, lr):
  
  name = 'mfcc_m_' + str(epochs) + '_dr' + str(dropout_rate) + '_wc' + str(weight_decay) + '_lr' + str(try_lr)
  return name +'.pt', name +'.txt'

In [18]:
m, t_file = fit(epochs = 500, 
                train_bs = 100, 
                n_train = n_train, 
                validate_bs = 200, 
                n_validate = n_validate,
                dropout_rate = 0.5, 
                weight_decay = 0.01, 
                lr = 1e-5,
                tolerance = 0.05)

Epoch:  9  == Early Stop due to an increase in validation loss
Train Finished


In [19]:
#torch.save(m, model_dir + m_save_path)

torch.save(m, 'model/early_stop_mfcc_noise_trained.pt')
file = open('model/early_stop_mfcc_noise_trained.txt',"w") 
file.write(t_file)
file.close()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type SoundRecognition_CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
###### Make change under this this line########

###############################################


### run model with several sets of hyperparameters
### the models and training results are saved

model_dir = 'model/'
result_dir = 'result/'



wc_list = [1e-3, 1e-4, 1e-5]
dr_list = [0.75, 0.9] #[0, 0.1, 0.5, 0.75, 0.9] do dr of 0, 0.1, 0.5,  with wc of 1e-4, 1e-5 and all lr
lr_list = [1e-4, 1e-5, 1e-6]
epochs = 1000
#dropout_rate = 0.5
#weight_decay = 1e-6

for try_wc in wc_list:
  for try_dr in dr_list:
    for try_lr in lr_list:
      m_save_path, f_save_path = getName(epochs, try_dr, try_wc, try_lr)
      print('Training: ', m_save_path)
      m, t_file = fit(epochs = epochs, 
                      train_bs = 100, 
                      n_train = n_train, 
                      validate_bs = 200, 
                      n_validate = n_validate,
                      dropout_rate = try_dr, 
                      weight_decay = try_wc, 
                      lr = try_lr)

      torch.save(m, model_dir + m_save_path)
      file = open(result_dir + f_save_path,"w") 
      file.write(t_file)
      file.close()
  
  


  
  
  

    
    

Training:  mfcc_m_1000_dr0.75_wc0.001_lr0.0001.pt
Epoch:  10  | Loss:  tensor(14.4796, device='cuda:0')  | Accuracy:  tensor(0.8349, device='cuda:0')
Epoch:  20  | Loss:  tensor(14.9914, device='cuda:0')  | Accuracy:  tensor(0.8552, device='cuda:0')
Epoch:  30  | Loss:  tensor(14.0225, device='cuda:0')  | Accuracy:  tensor(0.8456, device='cuda:0')
Epoch:  40  | Loss:  tensor(11.7950, device='cuda:0')  | Accuracy:  tensor(0.8810, device='cuda:0')
Epoch:  50  | Loss:  tensor(9.0250, device='cuda:0')  | Accuracy:  tensor(0.9270, device='cuda:0')
Epoch:  60  | Loss:  tensor(6.6335, device='cuda:0')  | Accuracy:  tensor(0.9435, device='cuda:0')
Epoch:  70  | Loss:  tensor(4.9772, device='cuda:0')  | Accuracy:  tensor(0.9580, device='cuda:0')
Epoch:  80  | Loss:  tensor(3.5786, device='cuda:0')  | Accuracy:  tensor(0.9659, device='cuda:0')
Epoch:  90  | Loss:  tensor(3.2540, device='cuda:0')  | Accuracy:  tensor(0.9654, device='cuda:0')
Epoch:  100  | Loss:  tensor(2.9891, device='cuda:0')  

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type SoundRecognition_CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Training:  mfcc_m_1000_dr0.75_wc0.001_lr1e-05.pt
Epoch:  10  | Loss:  tensor(6.1362, device='cuda:0')  | Accuracy:  tensor(0.9045, device='cuda:0')
Epoch:  20  | Loss:  tensor(7.4403, device='cuda:0')  | Accuracy:  tensor(0.8875, device='cuda:0')
Epoch:  30  | Loss:  tensor(8.5669, device='cuda:0')  | Accuracy:  tensor(0.8858, device='cuda:0')
Epoch:  40  | Loss:  tensor(9.5794, device='cuda:0')  | Accuracy:  tensor(0.8938, device='cuda:0')
Epoch:  50  | Loss:  tensor(10.6385, device='cuda:0')  | Accuracy:  tensor(0.8995, device='cuda:0')
Epoch:  60  | Loss:  tensor(11.3259, device='cuda:0')  | Accuracy:  tensor(0.9122, device='cuda:0')
Epoch:  70  | Loss:  tensor(12.1258, device='cuda:0')  | Accuracy:  tensor(0.9076, device='cuda:0')
Epoch:  80  | Loss:  tensor(12.8869, device='cuda:0')  | Accuracy:  tensor(0.8954, device='cuda:0')
Epoch:  90  | Loss:  tensor(13.3337, device='cuda:0')  | Accuracy:  tensor(0.8876, device='cuda:0')
Epoch:  100  | Loss:  tensor(13.8162, device='cuda:0') 